In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore') # 計算警告を非表示

# データの準備

In [ ]:
df = pd.read_csv('ebina_kion.csv')
df.head()

In [ ]:
df.tail()

In [ ]:
kion = pd.Series(df['kion'], dtype='float')
kion.index = pd.to_datetime(df['Month'])

print(kion.index.dtype)
print(len(kion))
kion.head()

In [ ]:
plt.figure(figsize=(12,8))
kion.plot()

# データの分析

In [ ]:
#分解
res = sm.tsa.seasonal_decompose(kion)

original = kion # オリジナルデータ
trend = res.trend # トレンドデータ
seasonal = res.seasonal # 季節性データ
residual = res.resid # 残差データ

plt.figure(figsize=(8, 8)) # グラフ描画枠作成、サイズ指定

# オリジナルデータのプロット
plt.subplot(411) # グラフ4行1列の1番目の位置（一番上）
plt.plot(original)
plt.ylabel('Original')

# trend データのプロット
plt.subplot(412) # グラフ4行1列の2番目の位置
plt.plot(trend)
plt.ylabel('Trend')

# seasonalデータ のプロット
plt.subplot(413) # グラフ4行1列の3番目の位置
plt.plot(seasonal)
plt.ylabel('Seasonality')

# residual データのプロット
plt.subplot(414) # グラフ4行1列の4番目の位置（一番下）
plt.plot(residual)
plt.ylabel('Residuals')

plt.tight_layout()

In [ ]:
#月別平均
kion.groupby(kion.index.month).mean()

In [ ]:
kion_month_mean = kion.groupby(kion.index.month).mean()
kion_month_mean.plot(kind='bar')

In [ ]:
#コレログラム
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(kion.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(kion, lags=40, ax=ax2)

# モデル推定

In [ ]:
# 単位根検定
adf_result = sm.tsa.stattools.adfuller(kion)
adf_result
# 棄却→単位根でない

In [ ]:
# 自動ARMAパラメータ推定関数
res_selection = sm.tsa.arma_order_select_ic(kion, max_ar=3, max_ma=3, ic='aic', trend='nc')
res_selection

In [ ]:
# SARIMAモデル作成その１
sarimax = sm.tsa.SARIMAX(kion, 
                        order=(3, 0, 3),
                        seasonal_order=(3, 0, 3, 12),
                        enforce_stationarity = False,
                        enforce_invertibility = False
                        ).fit()

In [ ]:
sarimax_resid = sarimax.resid # モデルの残差成分

# モデル残差のコレログラム
fig = plt.figure(figsize=(8, 8))

# 自己相関(ACF)のグラフ
ax1 = fig.add_subplot(211)
sm.graphics.tsa.plot_acf(sarimax_resid, lags=40, ax=ax1) #ACF計算とグラフ自動作成

# 偏自己相関(PACF)のグラフ
ax2 = fig.add_subplot(212)
sm.graphics.tsa.plot_pacf(sarimax_resid, lags=40, ax=ax2) #PACF計算とグラフ自動作成

plt.tight_layout() # グラフ間スキマ調整

In [ ]:
# SARIMAモデル（試しに）季節調整なし
sarimax_noseasonal = sm.tsa.SARIMAX(kion, 
                        order=(3, 0, 3),
                        seasonal_order=(3, 0, 3, 0),
                        enforce_stationarity = False,
                        enforce_invertibility = False
                        ).fit()

sarimax_noseasonal_resid = sarimax_noseasonal.resid # 残差成分

fig = plt.figure(figsize=(8, 8))

# 自己相関(ACF)のグラフ
ax1 = fig.add_subplot(211)
sm.graphics.tsa.plot_acf(sarimax_noseasonal_resid, lags=40, ax=ax1) #ACF計算とグラフ自動作成

# 偏自己相関(PACF)のグラフ
ax2 = fig.add_subplot(212)
sm.graphics.tsa.plot_pacf(sarimax_noseasonal_resid, lags=40, ax=ax2) #PACF計算とグラフ自動作成

plt.tight_layout() # グラフ間スキマ調整

In [ ]:
print(sarimax.aic) # 季節調整あり
print(sarimax_noseasonal.aic) # 季節調整なし

In [ ]:
sarimax.summary()

In [ ]:
sarimax_pred = sarimax.predict('1997-01', '2019-1') 

plt.figure(figsize=(12, 8))

#元データ
plt.plot(kion, label="original")
#モデル
plt.plot(sarimax_pred, c="b", label="model-pred", alpha=0.7)
plt.legend(loc='best')

In [ ]:
#先5年まで予測
sarimax_pred = sarimax.predict('2019-01', '2024-1') 

plt.figure(figsize=(12, 8))

plt.plot(kion, label="original")
plt.plot(sarimax_pred, c="b", label="model-pred", alpha=0.7)
plt.legend(loc='best')

# train & predict

In [ ]:
kion_train = kion['1997-01':'2015-12'] # モデル作成用データ 18年分
print(kion_train.head())
print(kion_train.tail())

In [ ]:
kion_test = kion['2015-01':'2019-1'] # テスト用データ4年分
print(kion_test.head())
print(kion_test.tail())

In [ ]:
# SRIMAモデル（テストデータを4年分除いてモデル作成）
sarimax_train = sm.tsa.SARIMAX(kion_train, 
                        order=(3, 0, 3),
                        seasonal_order=(3, 0, 3, 12),
                        enforce_stationarity = False,
                        enforce_invertibility = False
                        ).fit()

In [ ]:
sarimax_train_pred = sarimax_train.predict('2015-01', '2019-1') # テストデータ4年分予測

plt.figure(figsize=(12, 8))

plt.plot(kion, label="original")
plt.plot(sarimax_train_pred, c="b", label="model-pred", alpha=0.7)
plt.legend(loc='best')

In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(kion_test, label="actual") # 正解
plt.plot(sarimax_train_pred, c="b", label="predict", alpha=0.7) # 予測
plt.legend(loc='best')

In [ ]:
predict_dy = sarimax_train.get_prediction(start ='2015-12', end='2019-1')
type(predict_dy)

In [ ]:
predict_dy_ci = predict_dy.conf_int(alpha=0.05) #95%信頼区間
type(predict_dy_ci)

In [ ]:
predict_dy_ci # lower, upper取得

In [ ]:
predict_dy.predicted_mean # mean取得

In [ ]:
plt.figure(figsize=(8, 4))

plt.plot(kion_test, label="Actual")
plt.plot(predict_dy.predicted_mean, c="b", label="model-pred", alpha=0.7)

plt.fill_between(predict_dy_ci.index, predict_dy_ci.iloc[:, 0], predict_dy_ci.iloc[:, 1], color='g', alpha=0.2)
plt.legend(loc='upper left')

In [ ]:
predict2_dy = sarimax_train.get_prediction(start ='2015-12', end='2024-1') # 5年分の未来予測

predict2_dy_ci = predict2_dy.conf_int()

plt.figure(figsize=(12, 8))
plt.plot(kion['2012-01':], label="actual")
plt.plot(predict2_dy.predicted_mean, c="b", linestyle='--', label="model-pred", alpha=0.7)

plt.fill_between(predict2_dy_ci.index, predict2_dy_ci.iloc[:, 0], predict2_dy_ci.iloc[:, 1], color='g', alpha=0.2)
plt.legend(loc='upper left')